In [1]:
# Obtain parameters in Tidal Cases
import xarray as xr
import numpy as np
import os
import matplotlib.pyplot as plt

In [2]:
# Melting flux at ice front
def IFA(case_path, case_id):
    # Grid areas    
    Area = np.empty([90, 10])
    Area[:20,:] = 400
    Area[20:50,:] = 800
    Area[50:,:] = 1200

    file0 = xr.open_dataset(case_path + '/icefrntA_' + str(format(case_id,'03d')) + '.nc')
    t0 = 89
    #tn = 120
    # removed duplicated data caused by model restart
    file = file0.isel(T=~file0.get_index("T").duplicated())
    state = file.isel(Y=range(35,45), T=range(t0,len(file.T)))
    
    MR = state.icefrntA.isel(X=1).data.mean(0) # Melt rate at the icefront
    Qsm = (MR*Area).sum()/(24*3600)
    return Qsm

In [3]:
# Water temperature at ice front
def icefrntT(case_path, case_id):
    File1 = xr.open_dataset(case_path + '/state_' + str(format(case_id,'03d')) + '.nc')
    Grid = xr.open_dataset(case_path + '/grid_' + str(format(case_id,'03d')) + '.nc')  
    State = File1.isel(T=~File1.get_index("T").duplicated())
    
    # Confine to the range of fjord
    state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
    grid = Grid.isel(X=range(260), Xp1=range(261), Y=range(35,45))

    U = (state.U.data[:,:,:,1:] + state.U.data[:,:,:,:-1]) / 2 # Along-channel velocity
    
    drF = np.broadcast_to(grid.drF.data[np.newaxis, :, np.newaxis, np.newaxis], U.shape)
    dyF = np.broadcast_to(grid.dyF.data[np.newaxis, np.newaxis, :, :], U.shape)
    HFacC = np.broadcast_to(grid.HFacC.data[np.newaxis, :, :, :], U.shape)
    DA = drF * dyF * HFacC
   
    t0 = 89 # Starting time index
    da = DA[t0:,:,:,1].mean(axis=(0,2))
    Ti = state.Temp.data[t0:,:,:,1].mean(axis=(0,2))
    Ui = U[t0:,:,:,1].mean(axis=(0,2))
    
    Tvi = np.sum(Ti*da) / np.sum(da)
    return Tvi

In [4]:
path1 = '/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs'
path2 = '/work/oceans/wbao/MITgcm_results/iceplume/weak_Tide_minhs' # Qsg=250


caseN1 = np.array([1,2,3,4,5])
Qsm1 = np.empty(len(caseN1))
Tice1 = np.empty(len(caseN1))

caseN2 = np.array([1,2,3])
Qsm2 = np.empty(len(caseN2))
Tice2 = np.empty(len(caseN2))

for i in range(len(caseN1)):
    
    Qsm1[i] = IFA(path1, caseN1[i])
    Tice1[i] = icefrntT(path1, caseN1[i])

    
for j in range(len(caseN2)):
    
    Qsm2[j] = IFA(path2, caseN2[j])
    Tice2[j] = icefrntT(path2, caseN2[j])

In [5]:
Qsm2, Qsm1, Tice2, Tice1

(array([10.50572368, 10.50575029, 10.50595349]),
 array([10.50802133, 10.513916  , 10.52046552, 10.52912766, 10.53589816]),
 array([9.22971705, 9.2296666 , 9.22960501]),
 array([9.22936504, 9.22905646, 9.22889948, 9.22878582, 9.22874417]))

In [6]:
file1 = xr.open_dataset('/home/1959/Parameters/FjordModeling/4_wTide_minhs.nc')
file2 = xr.open_dataset('/home/1959/Parameters/FjordModeling/4_sTide_minhs.nc')

a11 = np.concatenate([file1.alpha11.data, file2.alpha11.data[:2]])
a22 = np.concatenate([file1.alpha22.data, file2.alpha22.data[:2]])

Qin1 = np.concatenate([file1.Qin1.data, file2.Qin1.data[:2]])
Qin1_adj = np.concatenate([file1.Qin1_adj.data, file2.Qin1_adj.data[:2]])

Qin2 = np.concatenate([file1.Qin2.data, file2.Qin2.data[:2]])
Qin2_adj = np.concatenate([file1.Qin2_adj.data, file2.Qin2_adj.data[:2]])

Qout1 = np.concatenate([file1.Qout1.data, file2.Qout1.data[:2]])
Qout1_adj = np.concatenate([file1.Qout1_adj.data, file2.Qout1_adj.data[:2]])

Qout2 = np.concatenate([file1.Qout2.data, file2.Qout2.data[:2]])
Qout2_adj = np.concatenate([file1.Qout2_adj.data, file2.Qout2_adj.data[:2]])

Ts_in = np.concatenate([file1.Ts_in.data, file2.Ts_in.data[:2]])
Ts = np.concatenate([file1.Ts.data, file2.Ts.data[:2]])
Tf = np.concatenate([file1.Tf.data, file2.Tf.data[:2]])

Qsm = np.concatenate([file1.Qsm.data, file2.Qsm.data[:2]])

In [35]:
#(Qin1_adj+Qout1_adj)/2, (Qin2+Qout2)/2, (Qin1-Qout2), (Qin1-Qout2)/Qin1*100
a11, a22, (Qout1-Qin2)/Qout1

(array([0.714578  , 0.71510556, 0.71483592, 0.72686785, 0.74121893]),
 array([0.04379473, 0.04338967, 0.04361214, 0.16954694, 0.27852152]),
 array([0.7228594 , 0.72359805, 0.72340441, 0.69702291, 0.67211946]))

In [15]:
Ts/Ts_in*100, Qsm

(array([95.93566617, 95.93500417, 95.93385365, 95.92938498, 95.91650881]),
 array([9.91612748, 9.91593574, 9.91578743, 9.91676165, 9.92117721]))

In [16]:
file3 = xr.open_dataset('/home/1959/Parameters/FjordModeling/1_BaseCase.nc')
a11b = file3.alpha11.data
Qin1b = file3.Qin1.data
Qin2b = file3.Qin2.data
Qout1b = file3.Qout1.data
Qout2b = file3.Qout2.data
Qsmb = file3.Qsm.data
Tsb = file3.Ts.data
Ts_inb = file3.Ts_in.data

In [18]:
a11b, (Qin2b+Qout2b)/2, Qin1b-Qout2b, Tsb/Ts_inb*100, Qsmb

(array([0.71530895, 0.56250147, 0.42047409, 0.29042472, 0.17951959,
        0.01136236]),
 array([2548.01896578, 3872.47116847, 4963.0527807 , 5844.6496557 ,
        6406.33548218, 7160.25633201]),
 array([6352.33882864, 5071.81799796, 3838.21230657, 2775.84609344,
        1630.28540874,  -64.18643813]),
 array([95.93627934, 97.44680653, 98.30763666, 98.88812863, 99.27479804,
        99.99391302]),
 array([ 9.91626286, 10.31809511, 10.35915423, 10.51910725, 10.86113773,
         9.29910293]))

In [31]:
case_path = '/work/oceans/wbao/MITgcm_results/iceplume/6_Tide_minhs'
case_id = 5
State0 = xr.open_dataset(case_path + '/state_' + str(format(case_id,'03d')) + '.nc')
State = State0.isel(T=~State0.get_index("T").duplicated())
state = State.isel(X=range(260), Xp1=range(261), Y=range(35,45), T=range(1,len(State.T)))
eta = state.Eta.data

In [32]:
eta.max(), eta.min(), eta.mean()

(1.1163096, 0.0, 0.54295653)